In [ ]:
import scipp as sc
import numpy as np

In [ ]:
#import sys
#sys.path.append('/home/simon/code/ess-legacy/sans')
#import dataconfig # run make_config.py to create this
#from loki import LoKI
#loki = LoKI()
#logical = loki.to_logical_dims(data)
#logical.to_hdf5(filename='loki-at-larmor-reshaped.hdf5')

# Part 2: Working with masks

## Introduction

Scipp supports non-destructive masks stored alongside data.
In this part we learn how to create and use masks.
As a side effect, the exercises will help in getting more familiar with the basic concepts of operations.

We use the same data as in Part 1:

In [ ]:
data = sc.io.open_hdf5(filename='loki-at-larmor.hdf5')
counts = sc.sum(data, 'tof') # used later
data

Masks are variables with `dtype=bool`, stored in the `masks` dict of a data array.
The result of comparison between variables can thus be used as masks:

In [ ]:
data.coords['spectrum'] < sc.scalar(100)

## Exercise 1: Masking a prompt pulse

1. Create a promt-pulse mask for the region between $17500~\mathrm{\mu s}$ and $19000~\mathrm{\mu s}$.
   Notes:
   - Use comparison operators such as `==`, `<=` or `>`.
   - Combine multiple conditions into one using `&` ("and"), `|` ("or"), or `^` ("exclusive or").
   - Masks are stored in a data array by storing them in the `masks` dictionary, e.g., `data.masks['promt-pulse'] = ...`.
   - If something goes wrong, masks can be removed with Python's `del`, e.g., `del data.masks['wrong']`.
   - If you run into an error regarding a length mismatch when inserting the coordinate, remember that `'tof'` is a bin-edge coordinate, i.e., it is by 1 longer than the number of bins.
     Use, e.g., only the left bin edges, i.e., all but the last, to create the masks.
2. Use the HTML view and plot the data after masking to explore the effect.
3. Pass a `dict` containing `counts` (computed above as `counts = sc.sum(data, 'tof')`) and the equivalent counts computed *after* masking to `sc.plot.plot`.
   Use this to verify that the promt-pulse mask results in removal of counts.

In [ ]:
tof = data.coords['tof']
data.masks['promt-pulse'] = (tof['tof',:-1] > 17500.0 * sc.units.us) & (tof['tof',1:] < 19000.0 * sc.units.us)
sc.plot.plot({'before':counts, 'after':sc.sum(data, 'tof')})

## Exercise 2: Masking spatially

By masking an `x` range, mask the end of the tubes.
- Define `x = sc.geometry.x(data.coords['position'])` to extract only the x-component of the position coordinate.
- Create the masks.
- Use the instrument view (`sc.neutron.instrumentview(data)`) to inspect the result.

In [ ]:
x = sc.geometry.x(data.coords['position'])
data.masks['tube-ends'] = x < -0.2 * sc.units.m
sc.neutron.instrument_view(data)

## Exercise 3: Combining conditions

Mask the broken pixels with zero counts near the beam stop (center).
- Note that there are pixels at larger scattering angles (larger x) which have real zeros.
  These should not be masked.
- Combine the condition for zero counts with a spatial mask, e.g., based on `x`, to ensure the masks takes only effect close to the direct beam / beam stop.

In [ ]:
# This would mask too much, what needs to be added?
counts.data == 0.0 * sc.units.counts

In [ ]:
broken = (counts.data == 0.0 * sc.units.counts) & (sc.abs(x) < 0.1 * sc.units.m)
data.masks['bad-pixels'] = broken
sc.neutron.instrument_view(data, pixel_size=0.01)

## Exercise 4: More spatial masking

Pick one (or more, if desired):

- Mask a "circle" (in $x$-$y$ plane, i.e., a cylinder aligned with $\hat z$)
- Mask a ring based on $x$ and $y$
- Mask a scattering-angle ($\theta$) range.
  Hint: The scattering angle can be computed as `theta = sc.neutron.scattering_angle(data)`
- Mask a wedge (pick one).
  Hint: `phi = sc.atan2(y,x)`

In [ ]:
pos = data.coords['position']
x = sc.geometry.x(pos)
y = sc.geometry.y(pos)
z = sc.geometry.z(pos)

# could use offsets x0 and y0 to mask away from z axis
r = sc.sqrt(x*x + y*y)
data.masks['circle'] = r < 0.09*sc.units.m

data.masks['ring'] = (0.14*sc.units.m < r) & (r < 0.19*sc.units.m)

theta = sc.neutron.scattering_angle(data)
data.masks['theta'] = (0.03*sc.units.rad < theta) & (theta < 0.04*sc.units.rad)

phi = sc.atan2(y,x) * ((180.0 * sc.units.deg) / (np.pi * sc.units.rad))
data.masks['wedge'] = (10.0*sc.units.deg < phi) & (phi < 20.0*sc.units.deg)

sc.neutron.instrument_view(data, pixel_size=0.01)

## Bonus exercise: Generalize techniques learned for masking for grouping

- Adapt the code for masking a wedge to return an integer sector index (e.g, 0...5).
- Store the result as a coordinate.
- Use `groupby` to group by sector.
  Note that `sc.groupby(...).copy(group)` can be used to extract a given group by index, instead of applying reductions.

## Masks in (grouped) reduction operations

Finally, let us group according to scattering angle and sum spectra.
Questions:
- Can you see the effect of the circle/ring/theta-range that you masked above?
- Why is the promt-pulse mask preserved, but not the other masks?

In [ ]:
theta_edges = sc.array(dims=['theta'], unit='rad', values=np.linspace(0,0.1, num=100))
data.coords['theta'] = sc.neutron.scattering_angle(data)
sc.groupby(data, group='theta', bins=theta_edges).sum('spectrum').plot()